<a href="https://colab.research.google.com/github/profitter261/Garbage-Image-Classification-Using-DL/blob/main/best_model_for_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing neccesary Libraries and Dataset

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification?utm_source=chatgpt.com&select=Garbage+classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: arv03arv
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification


100%|██████████| 82.0M/82.0M [00:00<00:00, 1.20GB/s]

In [ ]:
import os
os.listdir("/content/garbage-classification/Garbage classification/Garbage classification")

['cardboard', 'plastic', 'glass', 'trash', 'metal', 'paper']

# Test-Train split and Model Training

In [ ]:
import os

DATA_DIR = "/content/garbage-classification/Garbage classification/Garbage classification"  # adjust path

valid_ext = [".jpg", ".jpeg", ".png", ".bmp", ".gif"]

for root, dirs, files in os.walk(DATA_DIR):
    for f in files:
        if not f.lower().endswith(tuple(valid_ext)):
            print("Removing invalid file:", os.path.join(root, f))
            os.remove(os.path.join(root, f))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

data_dir = "/content/garbage-classification/Garbage classification/Garbage classification"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
VAL_SPLIT = 0.2

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VAL_SPLIT,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

NUM_CLASSES = len(train_gen.class_indices)

Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [ ]:
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=IMG_SIZE+(3,))
base_model.trainable = True  # unfreeze all layers for fine-tuning

# Optional: Freeze first few layers, fine-tune only last blocks
for layer in base_model.layers[:-50]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
preds = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=preds)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Compiling and Fitting the Model

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint("InceptionV3_best.h5", monitor="val_loss", save_best_only=True)

In [ ]:
history = model.fit(
    train_gen,
    epochs=20,  # full fine-tuning
    validation_data=val_gen,
    callbacks=[early_stop, checkpoint]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.2048 - loss: 1.9204

64/64 ━━━━━━━━━━━━━━━━━━━━ 363s 5s/step - accuracy: 0.2063 - loss: 1.9173 - val_accuracy: 0.5487 - val_loss: 1.2266
Epoch 2/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5546 - loss: 1.2733

64/64 ━━━━━━━━━━━━━━━━━━━━ 338s 5s/step - accuracy: 0.5553 - loss: 1.2718 - val_accuracy: 0.6302 - val_loss: 0.9924
Epoch 3/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6810 - loss: 0.9797

64/64 ━━━━━━━━━━━━━━━━━━━━ 341s 5s/step - accuracy: 0.6811 - loss: 0.9791 - val_accuracy: 0.6938 - val_loss: 0.8845
Epoch 4/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7335 - loss: 0.8230

...

64/64 ━━━━━━━━━━━━━━━━━━━━ 333s 5s/step - accuracy: 0.8475 - loss: 0.4368 - val_accuracy: 0.7734 - val_loss: 0.6047
Epoch 12/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8614 - loss: 0.4348

64/64 ━━━━━━━━━━━━━━━━━━━━ 332s 5s/step - accuracy: 0.8613 - loss: 0.4349 - val_accuracy: 0.7813 - val_loss: 0.5875
Epoch 13/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 329s 5s/step - accuracy: 0.8669 - loss: 0.4116 - val_accuracy: 0.8072 - val_loss: 0.5946
Epoch 14/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 331s 5s/step - accuracy: 0.8806 - loss: 0.3679 - val_accuracy: 0.7813 - val_loss: 0.6200
Epoch 15/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8864 - loss: 0.3599

64/64 ━━━━━━━━━━━━━━━━━━━━ 354s 6s/step - accuracy: 0.8863 - loss: 0.3600 - val_accuracy: 0.8072 - val_loss: 0.5622
Epoch 16/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8902 - loss: 0.3440

64/64 ━━━━━━━━━━━━━━━━━━━━ 332s 5s/step - accuracy: 0.8902 - loss: 0.3438 - val_accuracy: 0.8072 - val_loss: 0.5516
Epoch 17/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 355s 6s/step - accuracy: 0.9005 - loss: 0.3222 - val_accuracy: 0.7913 - val_loss: 0.6058
Epoch 18/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9093 - loss: 0.2987

64/64 ━━━━━━━━━━━━━━━━━━━━ 331s 5s/step - accuracy: 0.9093 - loss: 0.2988 - val_accuracy: 0.8171 - val_loss: 0.5495
Epoch 19/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 356s 6s/step - accuracy: 0.9119 - loss: 0.2860 - val_accuracy: 0.8091 - val_loss: 0.5797
Epoch 20/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 333s 5s/step - accuracy: 0.9151 - loss: 0.2820 - val_accuracy: 0.8211 - val_loss: 0.5522


# Model Tuning and optimization

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # freeze base layers

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(6, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=output)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Model Saving

In [ ]:
# Already saved best weights via ModelCheckpoint
# You can also save the full model again:
model.save("InceptionV3_final.h5")
print("Model saved as InceptionV3_final.h5")

Model saved as InceptionV3_final.h5
